In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from datetime import datetime, timedelta

In [ ]:
Fireworks = pd.read_csv('FireworksMaster03_24.csv')
Fireworks['Date of Incident'] = pd.to_datetime(Fireworks['Date of Incident'])
FY23FW = Fireworks[Fireworks['Date of Incident'] > '2023-08-11']
ZoneCuts = pd.read_excel('Firework Zone Cuts.xlsx')

In [ ]:
MKFireworks = FY23FW[FY23FW['Division'] == 'Magic Kingdom Park']
ECFireworks = FY23FW[FY23FW['Division'] == 'Epcot']
ZoneCuts = ZoneCuts[ZoneCuts['Performance Profile Response'] == 'Yes']

In [ ]:
start_date = datetime(2023, 8, 11)
end_date = datetime.now()

date_range = pd.date_range(start=start_date, end=end_date, freq='D')

df = pd.DataFrame({'Date': date_range, 'Firework_Incident': 'no'})


NotSoScary = ['2023-08-11', '2023-08-15', '2023-08-18', '2023-08-22', '2023-08-25', '2023-08-29', '2023-09-01'
              , '2023-09-01', '2023-09-04', '2023-09-08', '2023-09-10', '2023-09-12', '2023-09-15', '2023-09-17'
             , '2023-09-19', '2023-09-22', '2023-09-24', '2023-09-26', '2023-09-28', '2023-09-29', 
             '2023-10-1', '2023-10-03', '2023-10-05', '2023-10-06', '2023-10-09', '2023-10-10', '2023-10-12'
             , '2023-10-13', '2023-10-15', '2023-10-17', '2023-10-19', '2023-10-20', '2023-10-22', '2023-10-24'
             , '2023-10-26', '2023-10-27', '2023-10-29', '2023-10-31', '2023-11-1']

for date_str in NotSoScary:
    date_to_mark = pd.to_datetime(date_str)
    df.loc[df['Date'] == date_to_mark, 'Special_Event'] = 'Not So Scary'

#VeryMerry Dates
VeryMerry = ['2023-11-09', '2023-11-10', '2023-11-13', '2023-11-14', '2023-11-16', '2023-11-17', '2023-11-19'
                 , '2023-11-21', '2023-11-22', '2023-11-26', '2023-11-28', '2023-11-30', '2023-12-1', '2023-12-3'
                 , '2023-12-5', '2023-12-7', '2023-12-8', '2023-12-10', '2023-12-12', '2023-12-14', '2023-12-15'
                 , '2023-12-17', '2023-12-19', '2023-12-21', '2023-12-22']

for date_str in VeryMerry:
    date_to_mark = pd.to_datetime(date_str)
    df.loc[df['Date'] == date_to_mark, 'Special_Event'] = 'Very Merry'

In [ ]:
merged_df = pd.merge(MKFireworks, df, left_on='Date of Incident', right_on='Date', how='left')
merged_df['Special_Event'].fillna('NoEvent', inplace=True)

In [ ]:
count_df = merged_df.groupby('Special_Event')['Claim Number'].count().reset_index()
count_df.rename(columns={'Claim Number': 'Incident Count'}, inplace=True)

colors = ['#bcbcbc', '#9a9a9a', '#8e8e8e']

plt.bar(count_df['Special_Event'], count_df['Incident Count'], color=colors)
plt.xlabel('Special Event')
plt.ylabel('Incident Count')
plt.title('Incident Count vs. Special Events 8-11-2023 to 12-1-2023')
plt.show()

In [ ]:
merged_df

In [ ]:
merged_df['Time of Incident'] = pd.to_datetime(merged_df['Time of Incident'], format='%H%M', errors='coerce').dt.time

merged_df

In [ ]:
def determine_show(row):
    special_event = row['Special_Event']
    time_of_incident = row['Time of Incident']

    if pd.isnull(time_of_incident) or pd.isna(time_of_incident):
        return f'{special_event}, Unknown Show'

    if special_event == 'NoEvent':
        if time_of_incident < pd.to_datetime('20:00', format='%H:%M').time():
            return 'Magical Friendship Faire'
        else:
            return 'Happily Ever After'
    elif special_event == 'Not So Scary':
        if time_of_incident < pd.to_datetime('17:00', format='%H:%M').time():
            return 'Magical Friendship Faire'
        elif pd.to_datetime('17:00', format='%H:%M').time() <= time_of_incident < pd.to_datetime('22:00', format='%H:%M').time():
            return 'Not So Scary Stage Show'
        elif pd.to_datetime('22:00', format='%H:%M').time() <= time_of_incident <= pd.to_datetime('22:55', format='%H:%M').time():
            return 'Not So Scary Fireworks'
        else:
            return 'Not So Scary Stage Show'
    elif special_event == 'Very Merry':
        if time_of_incident < pd.to_datetime('17:00', format='%H:%M').time():
            return 'Magical Friendship Faire'
        elif pd.to_datetime('17:00', format='%H:%M').time() <= time_of_incident < pd.to_datetime('22:00', format='%H:%M').time():
            return 'Merriest Celebration'
        elif pd.to_datetime('22:00', format='%H:%M').time() <= time_of_incident <= pd.to_datetime('22:55', format='%H:%M').time():
            return 'Chrismastime Fireworks'
        else:
            return 'Merriest Celebration'
    else:
        return 'Unknown Show'

In [ ]:
merged_df['Show'] = merged_df.apply(determine_show, axis=1)

allFW = merged_df[['Claim Number', 'Date of Incident', 'Time of Incident', 'Special_Event', 'Show', 'LOCATION', 'GPS Coordinates', 'Incident Description']]

In [ ]:
allFW.to_csv('J:/Data/RMSA Analysis/0 - Team Working Files/10 - Nick/AllFWIncidents.csv', index=False)

In [ ]:
allFW

In [ ]:
result_df = pd.merge(ZoneCuts, merged_df, left_on='Date', right_on='Date of Incident', how='inner')

result_df[['Claim Number', 'Date of Incident', 'Time of Incident', 'Special_Event', 'Show', 'Show Name', 'Extended Land Closures']]

In [ ]:
result_df.to_csv('J:/Data/RMSA Analysis/0 - Team Working Files/10 - Nick/FireworksZoneClosures.csv', index=False)

In [ ]:
show_counts = merged_df['Show'].value_counts()

plt.figure(figsize=(10, 6))
show_counts.plot(kind='bar', color='skyblue')
plt.title('Counts of Each Show')
plt.xlabel('Show')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
show_counts = merged_df['Show'].value_counts()

sns.set(style="whitegrid")

plt.figure(figsize=(12, 6))
sns.barplot(x=show_counts.index, y=show_counts.values, palette="viridis")
plt.title('Counts of Each Show 8-11-2023 to 12-13-2023')
plt.xlabel('Show')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout() 
plt.show()

In [ ]:
show_counts_df = pd.DataFrame({'Show': show_counts.index, 'Count': show_counts.values})

# Add a title
table_title = "Counts of Each Show 8-11-2023 to 12-23-2023"

# Print the table using tabulate
table = tabulate(show_counts_df, headers='keys', tablefmt='fancy_grid', showindex=False)

In [ ]:
print(f"{table_title}\n")
print(table)

In [ ]:
show_counts_df = pd.DataFrame({'Show': show_counts.index, 'Count': show_counts.values})

# Add a title
table_title = "Counts of Each Show"

# Print the table in Markdown format
markdown_table = show_counts_df.to_markdown(index=False)

# Display the title and the table
print(f"# {table_title}\n")
print(markdown_table)

In [ ]:
incident_counts = merged_df['Date of Incident'].value_counts().sort_index()

# Set Seaborn style
sns.set(style="whitegrid")

# Create a line plot with incident dates and counts
plt.figure(figsize=(12, 6))
sns.lineplot(x=incident_counts.index, y=incident_counts.values, marker='o', color='skyblue')
plt.title('Incidents Over Time')
plt.xlabel('Incident Date')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
ZoneCuts